### Deploy Web App on Azure Container Services (AKS)
In this notebook, we will set up an Azure Container Service which will be managed by Kubernetes. We will then take the Docker image we created earlier that contains our app and deploy it to the AKS cluster. Then, we will check everything is working by sending an image to it and getting it scored.
    
The process is split into the following steps:
* [Define our resource names](#section1)
* [Login to Azure](#section2)
* [Create resource group and create AKS](#section3)
* [Connect to AKS](#section4)
* [Deploy our app](#section5)
* [Tear it all down](#section6)

This guide assumes is designed to be run on linux and requires that the Azure CLI is installed.

<a id='section1'></a>
## Setup
Below are the various name definitions for the resources needed to setup ACS as well as the name of the Docker image we will be using.

**Some outputs (and inputs) below have been hidden/masked for confidentiality**

In [1]:
# Please modify the below as you see fit
resource_group = "<RESOURCE_GROUP>" 
aks_name = "<AKS_CLUSTER_NAME>"
location = "eastus"

image_name = '<YOUR_DOCKER_IMAGE>' # 'masalvar/tfresnet-gpu' Feel free to use this Image if you want to 
                                   # skip creating your own container
selected_subscription = "'<YOUR SUBSCRIPTION>'" # If you have multiple subscriptions select 
                                                # the subscription you want to use here

<a id='section2'></a>
## Azure account login
The command below will initiate a login to your Azure account. It will pop up with an url to go to where you will enter a one off code and log into your Azure account using your browser.

In [ ]:
!az login -o table

In [3]:
!az account set --subscription $selected_subscription

In [ ]:
!az account show

In [5]:
!az provider register -n Microsoft.ContainerService

Registering is still on-going. You can monitor using 'az provider show -n Microsoft.ContainerService'


<a id='section3'></a>
## Create resource group and create AKS

### Create resource group
Azure encourages the use of groups to organise all the Azure components you deploy. That way it is easier to find them but also we can deleted a number of resources simply by deleting the group.

In [3]:
!az group create --name $resource_group --location $location

{
  "id": "/subscriptions/edf507a2-6235-46c5-b560-fd463ba2e771/resourceGroups/msaksrg",
  "location": "eastus",
  "managedBy": null,
  "name": "msaksrg",
  "properties": {
    "provisioningState": "Succeeded"
  },
  "tags": null
}


Below, we create the AKS cluster in the resource group we created earlier.

In [4]:
!az aks create --resource-group $resource_group --name $aks_name --node-count 1 --generate-ssh-keys -s Standard_NC6

{- Finished ..
  "additionalProperties": {},
  "agentPoolProfiles": [
    {
      "additionalProperties": {},
      "count": 1,
      "dnsPrefix": null,
      "fqdn": null,
      "name": "nodepool1",
      "osDiskSizeGb": null,
      "osType": "Linux",
      "ports": null,
      "storageProfile": "ManagedDisks",
      "vmSize": "Standard_NC6",
      "vnetSubnetId": null
    }
  ],
  "dnsPrefix": "msAKSTFClu-msaksrg-edf507",
  "fqdn": "msakstfclu-msaksrg-edf507-9dc6365c.hcp.eastus.azmk8s.io",
  "id": "/subscriptions/edf507a2-6235-46c5-b560-fd463ba2e771/resourcegroups/msaksrg/providers/Microsoft.ContainerService/managedClusters/msAKSTFCluster",
  "kubernetesVersion": "1.8.10",
  "linuxProfile": {
    "additionalProperties": {},
    "adminUsername": "azureuser",
    "ssh": {
      "additionalProperties": {},
      "publicKeys": [
        {
          "additionalProperties": {},
          "keyData": "ssh-rsa AAAAB3NzaC1yc2EAAAADAQABAAABAQDVfKBWPBKS84wluD3DJ0t3hepO2F13pz1VI5d4c7Tn4d80rSKJkF2

### Install kubectl CLI

To connect to the Kubernetes cluster, we will use kubectl, the Kubernetes command-line client. To install, run the following:

In [6]:
!sudo az aks install-cli

Please ensure that /usr/local/bin is in your search PATH, so the `kubectl` command can be found.


<a id='section4'></a>
## Connect to AKS cluster

To configure kubectl to connect to the Kubernetes cluster, run the following command:

In [5]:
!az aks get-credentials --resource-group $resource_group --name $aks_name

Merged "msAKSTFCluster" as current context in /home/mat/.kube/config


Let's verify connection by listing the nodes.

In [6]:
!kubectl get nodes

NAME                       STATUS    ROLES     AGE       VERSION
aks-nodepool1-27496346-0   Ready     agent     3m        v1.8.10


Let's check the pods on our cluster.

In [7]:
!kubectl get pods --all-namespaces

NAMESPACE     NAME                                    READY     STATUS    RESTARTS   AGE
kube-system   heapster-75f8df9884-vlt25               2/2       Running   0          1m
kube-system   kube-dns-v20-5bf84586f4-9jd9r           3/3       Running   0          1m
kube-system   kube-dns-v20-5bf84586f4-f8nsn           3/3       Running   0          1m
kube-system   kube-proxy-x64jp                        1/1       Running   0          1m
kube-system   kube-svc-redirect-mkwss                 1/1       Running   0          1m
kube-system   kubernetes-dashboard-665f768455-npsfh   1/1       Running   0          1m
kube-system   tunnelfront-5c48644fb8-4c6dt            1/1       Running   0          1m


<a id='section5'></a>
## Deploy application

Below we define our Kubernetes manifest file for our service and load balancer. Note that we have to specify the volume mounts to the drivers that are located on the node.


In [8]:
app_template = {
  "apiVersion": "apps/v1beta1",
  "kind": "Deployment",
  "metadata": {
      "name": "azure-dl"
  },
  "spec":{
      "replicas":1,
      "template":{
          "metadata":{
              "labels":{
                  "app":"azure-dl"
              }
          },
          "spec":{
              "containers":[
                  {
                      "name": "azure-dl",
                      "image": image_name,
                      "env":[
                          {
                              "name": "LD_LIBRARY_PATH",
                              "value": "$LD_LIBRARY_PATH:/usr/local/nvidia/lib64:/opt/conda/envs/py3.6/lib"
                          }
                      ],
                      "ports":[
                          {
                              "containerPort":80,
                              "name":"model"
                          }
                      ],
                      "volumeMounts":[
                          {
                            "mountPath": "/usr/local/nvidia",
                            "name": "nvidia"
                          }
                      ],
                      "resources":{
                           "requests":{
                               "alpha.kubernetes.io/nvidia-gpu": 1
                           },
                           "limits":{
                               "alpha.kubernetes.io/nvidia-gpu": 1
                           }
                       }  
                  }
              ],
              "volumes":[
                  {
                      "name": "nvidia",
                      "hostPath":{
                          "path":"/usr/local/nvidia"
                      },
                  },
              ]
          }
      }
  }
}

service_temp = {
  "apiVersion": "v1",
  "kind": "Service",
  "metadata": {
      "name": "azure-dl"
  },
  "spec":{
      "type": "LoadBalancer",
      "ports":[
          {
              "port":80
          }
      ],
      "selector":{
            "app":"azure-dl"
      }
   }
}

In [9]:
def write_json_to_file(json_dict, filename, mode='w'):
    with open(filename, mode) as outfile:
        json.dump(json_dict, outfile, indent=4,sort_keys=True)
        outfile.write('\n\n')

In [10]:
write_json_to_file(app_template, 'az-dl.json') # We write the service template to the json file

In [11]:
write_json_to_file(service_temp, 'az-dl.json', mode='a') # We add the loadbelanacer template to the json file

Let's check the manifest created.

In [12]:
!cat az-dl.json

{
    "apiVersion": "apps/v1beta1",
    "kind": "Deployment",
    "metadata": {
        "name": "azure-dl"
    },
    "spec": {
        "replicas": 1,
        "template": {
            "metadata": {
                "labels": {
                    "app": "azure-dl"
                }
            },
            "spec": {
                "containers": [
                    {
                        "env": [
                            {
                                "name": "LD_LIBRARY_PATH",
                                "value": "$LD_LIBRARY_PATH:/usr/local/nvidia/lib64:/opt/conda/envs/py3.6/lib"
                            }
                        ],
                        "image": "masalvar/tfresnet-gpu",
                        "name": "azure-dl",
                        "ports": [
                            {
                                "containerPort": 80,
                                "name": "model"
                            }
                        ],
            

Next, we will use kubectl create command to deploy our application.

In [13]:
!kubectl create -f az-dl.json

deployment.apps "azure-dl" created
service "azure-dl" created


Let's check if the pod is deployed.

In [14]:
!kubectl get pods --all-namespaces

NAMESPACE     NAME                                    READY     STATUS             RESTARTS   AGE
default       azure-dl-9db45b4f7-bqq5g                0/1       ImagePullBackOff   0          2m
kube-system   heapster-75f8df9884-vlt25               2/2       Running            0          4m
kube-system   kube-dns-v20-5bf84586f4-9jd9r           3/3       Running            0          5m
kube-system   kube-dns-v20-5bf84586f4-f8nsn           3/3       Running            0          5m
kube-system   kube-proxy-x64jp                        1/1       Running            0          5m
kube-system   kube-svc-redirect-mkwss                 1/1       Running            0          5m
kube-system   kubernetes-dashboard-665f768455-npsfh   1/1       Running            0          5m
kube-system   tunnelfront-5c48644fb8-4c6dt            1/1       Running            0          5m


If anything goes wrong you can use the commands below to observe the events on the node as well as review the logs.

In [15]:
!kubectl get events

LAST SEEN   FIRST SEEN   COUNT     NAME                                        KIND         SUBOBJECT                   TYPE      REASON                    SOURCE                                 MESSAGE
13m         13m          1         aks-nodepool1-27496346-0.152457f2d4c9a0c1   Node                                     Normal    Starting                  kubelet, aks-nodepool1-27496346-0      Starting kubelet.
11m         13m          8         aks-nodepool1-27496346-0.152457f2d68e13e7   Node                                     Normal    NodeHasSufficientDisk     kubelet, aks-nodepool1-27496346-0      Node aks-nodepool1-27496346-0 status is now: NodeHasSufficientDisk
11m         13m          8         aks-nodepool1-27496346-0.152457f2d68e3bbf   Node                                     Normal    NodeHasSufficientMemory   kubelet, aks-nodepool1-27496346-0      Node aks-nodepool1-27496346-0 status is now: NodeHasSufficientMemory
12m         13m          7         aks-nodepool1-27496346-

In [16]:
pod_json = !kubectl get pods -o json
pod_dict = json.loads(''.join(pod_json))
!kubectl logs {pod_dict['items'][0]['metadata']['name']}

2018-04-11 09:45:52,173 CRIT Supervisor running as root (no user in config file)
2018-04-11 09:45:52,175 INFO supervisord started with pid 1
2018-04-11 09:45:53,178 INFO spawned: 'program_exit' with pid 9
2018-04-11 09:45:53,179 INFO spawned: 'nginx' with pid 10
2018-04-11 09:45:53,180 INFO spawned: 'gunicorn' with pid 11
2018-04-11 09:45:54,211 INFO success: program_exit entered RUNNING state, process has stayed up for > than 1 seconds (startsecs)
2018-04-11 09:45:54.734234: I tensorflow/core/platform/cpu_feature_guard.cc:137] Your CPU supports instructions that this TensorFlow binary was not compiled to use: SSE4.1 SSE4.2 AVX AVX2 FMA
2018-04-11 09:45:58,739 INFO success: nginx entered RUNNING state, process has stayed up for > than 5 seconds (startsecs)
2018-04-11 09:46:01.556833: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1030] Found device 0 with properties: 
name: Tesla K80 major: 3 minor: 7 memoryClockRate(GHz): 0.8235
pciBusID: 1705:00:00.0
totalMemory: 11.17GiB freeMem

It can take a few minutes for the service to populate the EXTERNAL-IP field. This will be the IP you use to call the service. You can also specify an IP to use please see the AKS documentation for further details.

In [17]:
!kubectl get service azure-dl

NAME       TYPE           CLUSTER-IP   EXTERNAL-IP    PORT(S)        AGE
azure-dl   LoadBalancer   10.0.63.93   13.82.95.158   80:31941/TCP   7m


Now that we have our deployed service we can move onto [testing it](05_TestWebApp.ipynb)  
Below are the instructions to tear everything down once we are done with the cluster

<a id='section6'></a>
## Tear it all down 
Once you are done with your cluster you can use the following two commands to destroy it all.

In [18]:
!kubectl delete -f az-dl.json

deployment.apps "azure-dl" deleted
service "azure-dl" deleted


In [19]:
!az aks delete -n $aks_name -g $resource_group -y

inished ..

In [20]:
!az group delete --name $resource_group -y

inished ..